# Install Requirement

In [ ]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
# !pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets
# !pip3 install -q -U transformers==4.42.0
!pip3 install -q -U accelerate
!pip3 install -q -U transformers

In [ ]:
!huggingface-cli login --token YOUR_TOKEN_ID

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Load Library

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

import torch
import torch.nn as nn

import transformers
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)
from datasets import Dataset
from peft import LoraConfig, PeftConfig
import bitsandbytes as bnb

from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split

# Load Model

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [ ]:
model_name = "google/gemma-2-9b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    # quantization_config=bnb_config
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

# Map Task

In [ ]:
map_task = {'new_sum'  : 'Generate News Summarization',
'classify' : 'News Text Classification Category',
'headline' : 'Generate News Headline Summarization',
'tag' : 'Generate News Tags Suggestion'}

In [ ]:
USER_CHAT_TEMPLATE = '<start_of_turn>user\n{prompt}<end_of_turn>\n'
MODEL_CHAT_TEMPLATE = '<start_of_turn>model\n{prompt}<end_of_turn>\n'

# Create Inference Function

In [ ]:
def news_sum(text, task, model, tokenizer):
    base_prompt = 'Analyze the news article and and give a short summarize from the article within maximum 3 sentences in thai language.\n'
    text = text.replace(task, '')
    query = base_prompt + text
    prompt = (
    USER_CHAT_TEMPLATE.format(prompt=query)
    + '<start_of_turn>model\n'
)
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**input_ids, max_new_tokens=512, do_sample=True, temperature=0.4, num_beams=5)
    result = tokenizer.decode(outputs[0])

    return result

In [ ]:
def news_title(text, task, model, tokenizer):
    base_prompt = 'Analyze the news article and and generate news headline from the article within 1 sentences in thai language.\n'

    text = text.replace(task, '')
    query = base_prompt + text
    prompt = (
    USER_CHAT_TEMPLATE.format(prompt=query)
    + '<start_of_turn>model\n'
)
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**input_ids, max_new_tokens=512, do_sample=True, temperature=0.4, num_beams=5)
    result = tokenizer.decode(outputs[0])

    return result

In [ ]:
def news_tag(text, task, model, tokenizer):
    base_prompt = "Analyze the news article and generate keywords from news with maximum 5 tags in this format (tag1, tag2, tag3, tag4, tag5).\nAnswer in thai language except unique name."

    text = text.replace(task, '')
    query = base_prompt + text
    prompt = (
    USER_CHAT_TEMPLATE.format(prompt=query)
    + '<start_of_turn>model\n'
)
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**input_ids, max_new_tokens=512, do_sample=True, temperature=0.4, num_beams=5)
    result = tokenizer.decode(outputs[0])

    return result

In [ ]:
def news_clf(text, task, model, tokenizer):
    base_prompt = "You are multiclass classification model, Analyze the news and selected a single words only form provide list to classify main topic of the news: ['สังคม', 'ต่างประเทศ', 'กีฬา', 'การเมือง', 'อาชญากรรม', 'ศิลปะ-บันเทิง', 'ภูมิภาค', 'เศรษฐกิจ', 'วิทยาศาสตร์เทคโนโลยี', 'ภัยพิบัติ', 'สิ่งแวดล้อม']. do not add other context than words in list."

    # base_prompt = "ํYou are multiclass classification model, your role is to classfy query base on provide list ['สังคม', 'ต่างประเทศ', 'กีฬา', 'การเมือง', 'อาชญากรรม', 'ศิลปะ-บันเทิง', 'ภูมิภาค', 'เศรษฐกิจ', 'วิทยาศาสตร์เทคโนโลยี', 'ภัยพิบัติ', 'สิ่งแวดล้อม']. do not add other context than words in list."

    # base_prompt = "Analyze the news and classify it from one in list: ['สังคม', 'ต่างประเทศ', 'กีฬา', 'การเมือง', 'อาชญากรรม', 'ศิลปะ-บันเทิง', 'ภูมิภาค', 'เศรษฐกิจ', 'วิทยาศาสตร์เทคโนโลยี', 'ภัยพิบัติ', 'สิ่งแวดล้อม']. do not add other context than words in list."
    # base_prompt = "You are multiclass classification model, Analyze the news and selected a single words only form provide list to classify most of the news: ['สังคม', 'ต่างประเทศ', 'กีฬา', 'การเมือง', 'อาชญากรรม', 'ศิลปะ-บันเทิง', 'ภูมิภาค', 'เศรษฐกิจ', 'วิทยาศาสตร์เทคโนโลยี', 'ภัยพิบัติ', 'สิ่งแวดล้อม']. do not add other context than words in list."

    text = text.replace(task, '')
    query = base_prompt + text
    prompt = (
    USER_CHAT_TEMPLATE.format(prompt=query)
    + '<start_of_turn>model\n'
)
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**input_ids, max_new_tokens=512, do_sample=True, temperature=0.4, num_beams=5)
    result = tokenizer.decode(outputs[0])

    return result

## Analyze Function

In [ ]:
def analyze(task, text, model=model, tokenizer=tokenizer):
    map_task = {'news_sum'  : 'Generate News Summarization: ',
    'classify' : 'News Text Classification Category: ',
    'headline' : 'Generate News Headline Summarization: ',
    'tag' : 'Generate News Tags Suggestion: '}
    if task == 'classify':
        res = news_clf(text, map_task['classify'], model, tokenizer)
    elif task == 'news_sum':
        res = news_sum(text, map_task['news_sum'], model, tokenizer)
    elif task == 'headline':
        res = news_title(text, map_task['headline'], model, tokenizer)
    elif task == 'tag':
        res = news_tag(text, map_task['tag'], model, tokenizer)
    torch.cuda.empty_cache()

    return res

In [ ]:
categories = ['สังคม', 'ต่างประเทศ', 'กีฬา', 'การเมือง',
              'อาชญากรรม', 'ศิลปะ-บันเทิง', 'ภูมิภาค', 'เศรษฐกิจ',
              'วิทยาศาสตร์เทคโนโลยี', 'ภัยพิบัติ', 'สิ่งแวดล้อม']

# Inference

In [ ]:
query = """
OpenAI ประกาศซื้อกิจการ Multi (ชื่อเดิม Remotion) สตาร์ทอัปแพลตฟอร์มทำงานร่วมกัน (collaboration) ที่โฟกัสวิดีโอเป็นหลัก และเน้นลูกค้าองค์กร

รายงานบอกว่าดีลดังกล่าว OpenAI ไม่ได้ต้องการซื้อแพลตฟอร์มหรือเครื่องมือของ Multi แต่ต้องการซื้อตัวทีมงานพัฒนา ซึ่งมีอยู่ทั้งหมด 5 คน มาร่วมงานกับ OpenAI ซึ่งทำให้ Multi จะปิดให้บริการตั้งแต่ 24 กรกฎาคม เป็นต้นไป

Multi ก่อตั้งโดย Alexander Embiricos อดีตผู้จัดการฝ่ายผลิตภัณฑ์ที่ Dropbox โดยตอนนี้เป็นซีอีโอ และ Charley Ho อดีตวิศวกรซอฟต์แวร์ที่กูเกิล ซอฟต์แวร์ Multi มีจุดเด่นเช่น การแชร์หน้าจอพร้อมกันสูงสุด 10 คน ในระหว่างการประชุม รวมถึงการแชร์เอกสารข้อมูลต่าง ๆ

ดีลนี้เป็นการซื้อกิจการถัดมาของ OpenAI จากที่ประกาศซื้อสตาร์ทอัปด้านฐานข้อมูล Rockset ไปเมื่อวันก่อน
"""

In [ ]:
# query = """
# ผู้ว่าฯ สมุทรสาคร สั่งปิดพื้นที่ 4 จุด ที่พบกากแคดเมียมต่ออีก 30 วัน หลังผลตรวจ สำนักสิ่งแวดล้อมฯ พบคุณภาพดินภายใน-โดยรอบ เกินค่ามาตรฐาน สั่งฝ่ายเกี่ยวข้อง ลงพื้นที่ ตรวจสอบ 1 - 2 ก.ค.67
# วันนี้ (1 ก.ค.2567) ผู้สื่อข่าวรายงานว่า วันนี้ เวลา 10.00 น.คณะทำงานชุดปฏิบัติการด้านสารเคมีและวัตถุอันตรายตามคำสั่งกองอำนวยการป้องกันและบรรเทาสาธารณภัยจังหวัดสมุทรสาคร ได้แก่ อุตสาหกรรมจังหวัดสมุทรสาคร, พลังงานจังหวัดสมุทรสาคร, ผู้อำนวยการสำนักงานทรัพยากรธรรมชาติ และสิ่งแวดล้อมจังหวัดสมุทรสาคร , สวัสดิการและคุ้มครองแรงงานจังหวัดสมุทรสาคร และนายแพทย์สาธารณสุขจังหวัดสมุทรสาคร จะลงพื้นที่ ตรวจสอบสถานที่เก็บกากตะกอนแคดเมียม ที่โกดังใน ต.คลองมะเดื่อ อ.กระทุ่มแบน จ.สมุทรสาคร และอีกแห่ง คือ บ.ซิน หงส์ เฉิง อินเตอร์เทค 2008 (จำกัด) ในพื้นที่ ต.บางน้ำจืด อ.มือง จ.สมุทรสาคร

# นอกจากนี้ในวันพรุ่งนี้ (2 ก.ค.2567) จะตรวจสอบ ที่ บ.เจแอนด์ บี เมททอล จํากัด ในซอยกองพนันพล ต.บางน้ำจืด อ.เมืองสมุทรสาคร จ.สมุทรสาคร แห่งใหญ่ ที่เคยพบกากแคดเมียมมากที่สุดใน จ.สมุทรสาคร และโกดังอีกแห่งของ บ.เจ แอนด์ บี ในซอยเดียวกัน ที่อยู่ห่างออกไปประมาณ 150 ม.

# ผู้ว่าฯ สมุทรสาคร สั่งปิดพื้นที่ 4 จุด ที่พบกากแคดเมียมต่ออีก 30 วัน หลังผลตรวจ สำนักสิ่งแวดล้อมฯ พบคุณภาพดินภายใน-โดยรอบ เกินค่ามาตรฐาน สั่งฝ่ายเกี่ยวข้อง ลงพื้นที่ ตรวจสอบ 1 - 2 ก.ค.67
# ผู้ว่าฯ สมุทรสาคร สั่งปิดพื้นที่ 4 จุด ที่พบกากแคดเมียมต่ออีก 30 วัน หลังผลตรวจ สำนักสิ่งแวดล้อมฯ พบคุณภาพดินภายใน-โดยรอบ เกินค่ามาตรฐาน สั่งฝ่ายเกี่ยวข้อง ลงพื้นที่ ตรวจสอบ 1 - 2 ก.ค.67

# การลงตรวจในช่วงวันนี้ (1 ก.ค.2567) และวันพรุ่งนี้ (2 ก.ค.2567) สืบเนื่องมาจาก กองอำนวยการป้องกันและบรรเทาสาธารณภัยจังหวัดสมุทรสาคร ได้รายงานข้อมูลการขนย้าย กากตะกอนแคดเมียมภาพรวม เมื่อวันที่ 28 มิ.ย.2567 ระบุว่า แม้การขนย้ายกากตะกอนแคดเมียมมีกำหนดแล้วเสร็จในสิ้นเดือน มิ.ย.2567

# แต่จากการตรวจสอบของ สำนักสิ่งแวดล้อมและควบคุมมลพิษที่ 5 ซึ่งได้เข้าตรวจวัดคุณภาพดินภายในโรงงาน และบริเวณโดยรอบโรงงาน เจ แอนด์บี เมททอล จำกัด เมื่อวันที่ 6 มิ.ย.2567 ผลปรากฏว่า พบค่า โลหะหนักแคดเมียมในดินบริเวณหน้าอาคารเก็บกากตะกอนแคดเมียมจำนวน 12,084 มก.ต่อ กก. และพบค่าโลหะหนักแคดเมียมในดินบริเวณพื้นอาคารที่เก็บกากตะกอนแคดเมียมจำนวน 10,571 มก.ต่อ กก. ซึ่งเกินกว่า เกณฑ์การปนเปื้อนในดินที่กำหนดไว้ต้องไม่เกิน 810 มก.ต่อ กก.

# ด้านนายผล ดำธรรม ผู้ว่าราชการจังหวัดสมุทรสาคร ในฐานะผู้อำนวยการจังหวัด ตาม พ.ร.บ.ป้องกันและบรรเทาสาธารณภัย 2550 พิจารณาแล้ว เห็นว่า หากยังมีค่าสารโลหะหนักแคดเมียมปนเปื้อนในดินบริเวณสถานที่เก็บกากตะกอนแคดเมียม และพื้นที่ใกล้เคียงเกินเกณฑ์ที่กำหนด จะส่งผลกระทบและเป็นอันตรายแก่ชีวิต และร่างกายของประชาชนที่เข้าไปอยู่อาศัยหรือดำเนินกิจการใด ๆ ในบริเวณดังกล่าวได้

# นอกจากนี้ยังเห็นว่าความเสี่ยงที่จะเกิดสาธารณภัยในพื้นที่ดังกล่าวยังไม่หมดไป จึงประกาศขยายระยะเวลาห้ามบุคคลใด ๆ เข้าไปอยู่อาศัยหรือดำเนินกิจการใด ๆ ในพื้นที่ใกล้จะเกิดสาธารณภัยต่อเนื่อง จากประกาศเดิมไปอีกเป็นระยะเวลา 30 วัน โดยประกาศลงวันที่ 28 มิ.ย.2567

# ผู้ว่าฯ สมุทรสาคร สั่งปิดพื้นที่ 4 จุด ที่พบกากแคดเมียมต่ออีก 30 วัน หลังผลตรวจ สำนักสิ่งแวดล้อมฯ พบคุณภาพดินภายใน-โดยรอบ เกินค่ามาตรฐาน สั่งฝ่ายเกี่ยวข้อง ลงพื้นที่ ตรวจสอบ 1 - 2 ก.ค.67
# ผู้ว่าฯ สมุทรสาคร สั่งปิดพื้นที่ 4 จุด ที่พบกากแคดเมียมต่ออีก 30 วัน หลังผลตรวจ สำนักสิ่งแวดล้อมฯ พบคุณภาพดินภายใน-โดยรอบ เกินค่ามาตรฐาน สั่งฝ่ายเกี่ยวข้อง ลงพื้นที่ ตรวจสอบ 1 - 2 ก.ค.67

# พร้อมให้หน่วยงานที่เกี่ยวข้อง คณะทำงานชุดปฏิบัติการด้านสารเคมีและวัตถุอันตราย ดำเนินการตามแนวทางที่เคยให้ถือปฏิบัติ และเพื่อป้องกันไม่ให้เกิดความเสียหายในชีวิตและร่างกายของประชาชน จึงขอให้ดำเนินการใน 2 มาตรการ คือ

# 1.ให้คณะทำงานชุดปฎิบัติการฯ ลงตรวจสถานที่ ภายหลังการขนย้ายกากตะกอนแคดเมียมในวันที่ 1 ก.ค.2567 และ 2 ก.ค.2567 โดยให้ตรวจสอบว่า สถานที่ดังกล่าวมีความปลอดภัยต่อประชาชน สมควรจะประกาศยกเลิกการห้ามบุคคลใด ๆ เข้าไปอาศัยหรือดำเนินกิจการใด ๆ ในพื้นที่ดังกล่าวหรือไม่ แล้วรายงานผลการตรวจสอบให้กองอำนวยการป้องกันและบรรเทา ไป จ.สมุทรสาคร รับทราบภายในวันที่ 8 ก.ค.2567

# 2.ขอความร่วมมือสำนักงานทรัพยากรธรรมชาติและสิ่งแวดล้อมจังหวัดสมุทรสาคร และสำนักงานอุตสาหกรรมจังหวัดสมุทรสาครประสานหน่วยงานที่มีหน้าที่เข้าตรวจปริมาณการปนเปื้อนของสารแคดเมียมภายในสถานที่ตามประกาศข้างต้น และบริเวณใกล้เคียงแล้วส่งผลการตรวจให้กองอำนวยการป้องกันและบรรเทาสาธารณภัยจังหวัดสมุทรสาครรับทราบภายในวันที่ 8 ก.ค.2567

# ผู้สื่อข่าวรายงานว่า เมื่อวันที่ 28 มิ.ย.2567 ทางกองอำนวยการป้องกันและบรรเทาสาธารณภัยจังหวัดสมุทรสาคร ได้รายงานข้อมูลภาพรวม ปริมาณการขนย้ายกากแคดเมียม ใน จ.สมุทรสาคร ซึ่งถูกค้นพบมาก่อนหน้านี้ ใน 4 แห่ง และทยอยขนกลับไปยังต้นทาง จ.ตาก เสร็จเรียบร้อยแล้วทั้งหมด ทั้ง 4 แห่ง ได้แก่

# ผู้ว่าฯ สมุทรสาคร สั่งปิดพื้นที่ 4 จุด ที่พบกากแคดเมียมต่ออีก 30 วัน หลังผลตรวจ สำนักสิ่งแวดล้อมฯ พบคุณภาพดินภายใน-โดยรอบ เกินค่ามาตรฐาน สั่งฝ่ายเกี่ยวข้อง ลงพื้นที่ ตรวจสอบ 1 - 2 ก.ค.67
# ผู้ว่าฯ สมุทรสาคร สั่งปิดพื้นที่ 4 จุด ที่พบกากแคดเมียมต่ออีก 30 วัน หลังผลตรวจ สำนักสิ่งแวดล้อมฯ พบคุณภาพดินภายใน-โดยรอบ เกินค่ามาตรฐาน สั่งฝ่ายเกี่ยวข้อง ลงพื้นที่ ตรวจสอบ 1 - 2 ก.ค.67

# 1. บ.เจ แอนด์ บี เมททอล จำกัด ซึ่งเป็นโรงงานที่พบกากแคดเมียมมากที่สุดใน จ.สมุทรสาคร มีปริมาณแคดเมียมที่พบ จำนวน 6,151 ตัน ปริมาณที่ขนเสร็จแล้ว จำนวน 6,200.62 ตัน

# 2. บ.เจ แอนด์ บี เมททอล จำกัด อีกแห่งหนึ่ง ที่อยู่ใกล้กัน ปริมาณกากแคดเมียมที่พบ 717 ตันปริมาณที่ควรแล้วเสร็จ จำนวน 795.48 ตัน

# 3. บ.ซิน หงษ์ เฉิง อินเตอร์เทค 2008 จำกัด ปริมาณกากแคดเมียมที่พบ 1,005 ตัน ปริมาณที่ขนแล้ว จำนวน 1,013.23 ตัน

# 4.โกดังในพื้นที่หมู่ 7 ต.คลองมะเดื่อ อ.กระทุ่มแบน ปริมาณที่พบ 534 ตัน ปริมาณขน 552.10 ตัน เมื่อรวมทั้ง 4 แห่ง มีกากแคดเมียม ที่พบ จำนวน 8,407 ตัน มีปริมาณที่ขนย้ายเสร็จแล้ว จำนวน 8,561.43 ตัน
# """

In [ ]:
%%time
analyze('headline', query).split('<start_of_turn>model')[1].split('\n')[1].strip()

CPU times: user 5.21 s, sys: 216 ms, total: 5.42 s
Wall time: 5.41 s


'OpenAI ซื้อทีมพัฒนา Multi สตาร์ทอัปแพลตฟอร์มทำงานร่วมกันโฟกัสวิดีโอ'

In [ ]:
%%time
analyze('news_sum', query).split('<start_of_turn>model')[1].split('\n')[1].strip()

CPU times: user 17.4 s, sys: 311 ms, total: 17.7 s
Wall time: 17.7 s


'OpenAI ได้ประกาศซื้อกิจการ Multi, สตาร์ทอัปที่พัฒนาแพลตฟอร์มทำงานร่วมกันแบบโฟกัสวิดีโอ, โดยไม่ได้ซื้อแพลตฟอร์ม แต่ซื้อทีมพัฒนา 5 คนของ Multi.  Multi จะปิดให้บริการตั้งแต่วันที่ 24 กรกฎาคมเป็นต้นไป.  นี่เป็นการซื้อกิจการครั้งที่สองของ OpenAI ภายในระยะเวลาสั้นๆ หลังจากที่ซื้อ Rockset, สตาร์ทอัปด้านฐานข้อมูล, ไปเมื่อวานนี้.'

In [ ]:
%%time
analyze('tag', query).split('<start_of_turn>model')[1].split('\n')[1].strip()

CPU times: user 3.65 s, sys: 303 ms, total: 3.95 s
Wall time: 3.96 s


'(OpenAI, ซื้อกิจการ, Multi, วิดีโอ, องค์กร)'

In [ ]:
%%time
analyze('classify', query).split('<start_of_turn>model\n')[1].replace('<eos>', '').split('\n')[0].strip()

CPU times: user 2.25 s, sys: 379 ms, total: 2.63 s
Wall time: 2.65 s


'เศรษฐกิจ'